## Deploying Docker to Azure Web Apps

Based on Mathew's tutorial located [here](https://github.com/msalvaris/batch_shipyard_notebooks/blob/master/cifar_example/train_on_azure_batch_shipyard.ipynb)

Tutorial currently **not working**

In [1]:
import os 
import urllib
from os import path
import json
import requests
import time
from io import BytesIO
from PIL import Image, ImageOps
import base64

In [2]:
# Check that docker is working
!docker run --rm hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

5dd45222: Pull complete 971 B/971 BBDigest: sha256:c5515758d4c5e1e838e9cd307f6c6a0d620b5e07e6f927b07d05f6d12a1ac8d7
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more example

### Step 1. Create WebApp Code

In [3]:
%%bash
mkdir script
mkdir script/code

mkdir: cannot create directory ‘script’: File exists
mkdir: cannot create directory ‘script/code’: File exists


In [55]:
%%writefile script/code/model.py

import base64
import urllib
import numpy as np
import cntk
import pkg_resources
from flask import Flask, request
import json
from io import BytesIO
from PIL import Image, ImageOps
from cntk import load_model, combine

app = Flask(__name__)
print("Something outside of @app.route() is always loaded")

# Pre-load model
MODEL = load_model("ResNet_18.model")
print("Loaded model: ", MODEL)
# Pre-load labels
with open('synset-1k.txt', 'r') as f:
    LABELS = [l.rstrip() for l in f]
print(LABELS[:10])
print("Loaded {0} labels".format(len(LABELS)))

@app.route("/")
def healthy_me():
    return "healthy"

@app.route('/cntk')
def cntk_ver():
    return "CNTK version: {}".format(pkg_resources.get_distribution("cntk").version)

@app.route('/posttest', methods=['POST'])
def posttest():
    return "POST healthy"

@app.route("/api/uploader", methods=['POST'])
def api_upload_file():
    inputString = request.json['input']
    images = json.loads(inputString)   
    for base64ImgString in images:
        if base64ImgString.startswith('b\''):
            base64ImgString = base64ImgString[2:-1]
        base64Img = base64ImgString.encode('utf-8')
        # Preprocess the input data 
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        # Load image with PIL (RGB)
        img = Image.open(img_buffer).convert('RGB')
        img = ImageOps.fit(img, (224, 224), Image.ANTIALIAS)
        return json.dumps(run_some_deep_learning_cntk(img))

def run_some_deep_learning_cntk(rgb_pil_image):
    # Convert to BGR
    rgb_image = np.array(rgb_pil_image, dtype=np.float32)
    bgr_image = rgb_image[..., [2, 1, 0]]
    img = np.ascontiguousarray(np.rollaxis(bgr_image, 2))

    # Use last layer to make prediction
    z_out = combine([MODEL.outputs[3].owner])
    result = np.squeeze(z_out.eval({z_out.arguments[0]: [img]}))

    # Sort probabilities 
    a = np.argsort(result)[-1]
    predicted_category = " ".join(LABELS[a].split(" ")[1:])
    return predicted_category

if __name__ == '__main__':
    # This is just for debugging
    app.run(host='0.0.0.0', port=5005)

Overwriting script/code/model.py


In [187]:
%%writefile script/code/flaskconfig

server {
    listen 80;

    location = /favicon.ico { access_log off; log_not_found off; }

    location / {
        proxy_set_header Host $http_host;
        proxy_set_header X-Real-IP $remote_addr;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
        proxy_pass http://0.0.0.0:5005;
    }

}

Overwriting script/code/flaskconfig


In [ ]:
%%writefile script/code/start.sh
service nginx start
/usr/local/bin/gunicorn --workers 1 -m 007 --bind 0.0.0.0:5005 model:app

In [34]:
%%writefile script/code/requirements.txt
Flask
gunicorn
pillow

Overwriting script/code/requirements.txt


In [8]:
urllib.urlretrieve("https://bootstrap.pypa.io/get-pip.py", "script/code/get-pip.py")

('script/code/get-pip.py', <httplib.HTTPMessage instance at 0x7fb994279c68>)

In [9]:
urllib.urlretrieve('https://azurewebappcntk.blob.core.windows.net/model/ResNet_18.model', 'script/code/ResNet_18.model')

('script/code/ResNet_18.model',
 <httplib.HTTPMessage instance at 0x7fb994271b00>)

In [10]:
urllib.urlretrieve('https://azurewebappcntk.blob.core.windows.net/model/synset-1k.txt', 'script/code/synset-1k.txt')

('script/code/synset-1k.txt', <httplib.HTTPMessage instance at 0x7fb99428f368>)

### Step 2. LogIn

In [ ]:
!az login -o table
docker_registry = "XXXXXXXXXXX"
docker_registry_group = "XXXXXXXXXXXXX"
!az group create -n $docker_registry_group -l southcentralus -o table
!az acr create -n $docker_registry -g $docker_registry_group -l southcentralus -o table
!az acr update -n $docker_registry --admin-enabled true -o table
json_data = !az acr credential show -n $docker_registry
docker_username = json.loads(''.join(json_data))['username']
docker_password = json.loads(''.join(json_data))['password']
print(docker_username)
print(docker_password)
json_data = !az acr show -n $docker_registry
docker_registry_server = json.loads(''.join(json_data))['loginServer']

### Step 3. Create Docker Image

In [11]:
!mkdir script/docker

mkdir: cannot create directory ‘script/docker’: File exists


In [131]:
%%writefile script/docker/dockerfile

FROM ubuntu:16.04
RUN mkdir /code
WORKDIR /code
MAINTAINER Ilia Karmanov
ADD code /code
RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python3 \
        python3-dev \
        python3-setuptools \
        curl \
        nginx &&\
        python3 /code/get-pip.py && \
        rm /etc/nginx/sites-enabled/default && \
        cp /code/flaskconfig /etc/nginx/sites-available/ && \
        ln -s /etc/nginx/sites-available/flaskconfig /etc/nginx/sites-enabled/ && \
        python3 -m pip install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.0.beta15.0-cp35-cp35m-linux_x86_64.whl && \
        python3 -m pip install -r /code/requirements.txt && \
        chmod 777 /code/start.sh
EXPOSE 80
ENTRYPOINT /code/start.sh

Overwriting script/docker/dockerfile


In [204]:
container_name = docker_registry_server + "/ilkarman/liapp"
application_path = 'script'
docker_file_location = path.join(application_path, 'docker/dockerfile')

In [205]:
!docker login $docker_registry_server -u $docker_username -p $docker_password

In [ ]:
%%bash
docker stop $(docker ps -a -q)
docker rm $(docker ps -a -q)

In [ ]:
# Running from shell:
#docker_build = "docker build -t {0} -f {1} {2} --no-cache".format(container_name, docker_file_location, application_path)
docker_build

In [ ]:
# This will take a while; potentially run from shell instead to see output (there will be a lot)
!$docker_build

Test everything is working locally before pushing

In [209]:
# 1.23GB (ResNet_18.model is ~60MB)
#!docker images   

In [ ]:
# To debug
print(container_name)
# In shell (run interactive mode):
#docker run -p 8070:8090 -it $container_name
#conda info --env
#which python
# ... etc

In [211]:
test_cont = !docker run -p 80:80 -d $container_name
test_cont

['9d99096b62f19b0abb9abe7fa0991f6ba829bed26cf6075ad93fe149961f9757']

In [212]:
time.sleep(5)  # Wait to load

In [213]:
!curl http://0.0.0.0

healthy

In [214]:
!curl http://0.0.0.0/cntk

CNTK version: 2.0.beta15.0

In [215]:
IMAGEURL = "https://www.britishairways.com/assets/images/information/about-ba/fleet-facts/airbus-380-800/photo-gallery/240x295-BA-A380-exterior-2-high-res.jpg"

In [216]:
def url_img_to_json_img(url):
    bytfile = BytesIO(urllib.urlopen(url).read())
    img = Image.open(bytfile).convert('RGB')  # 3 Channels
    img = ImageOps.fit(img, (224, 224), Image.ANTIALIAS)  # Fixed size 
    imgio = BytesIO()
    img.save(imgio, 'PNG')
    imgio.seek(0)
    dataimg = base64.b64encode(imgio.read())
    return json.dumps(
        {'input':'[\"{0}\"]'.format(dataimg.decode('utf-8'))})

In [217]:
jsonimg = url_img_to_json_img(IMAGEURL)
jsonimg[:100]  # Example of json string
headers = {'content-type': 'application/json'}

requests.post('http://0.0.0.0/api/uploader', 
              data=jsonimg,
              headers=headers).json()

u'airliner'

In [218]:
!docker kill {test_cont[0]}

9d99096b62f19b0abb9abe7fa0991f6ba829bed26cf6075ad93fe149961f9757


### Step 4. Push Docker Image to Registry

In [ ]:
container_name

In [ ]:
!docker push $container_name

Note: the size of the image, since the Web App loads the container into RAM you would need to configure it to have more RAM than the size of the image (if this is not possible then try to remove unnecessary files in your image or use another).

Note: I'm not sure how much bigger deployed container is compared to compressed image?

### Step 5. Create Azure Web App from Docker Image

Note: Currently cannot find a CLI-way of doing this

1. Go to your Azure Portal and create a new 'Web App on Linux (preview)' resource

2. Click on 'Configure container' and select 'Private registry' under 'Image source'

3. Configure the 'App Service plan/Location' so that your Web App size has enough RAM to host the container (e.g. S3)

4. Enter the details to connect to your ACR:
```
Image and optional tag: ikmscontainer.azurecr.io/ilkarman/dockergunicorn
Server URL: http://ikmscontainer.azurecr.io
Login username:<docker_username>
Password:<docker_password>
```

6. Create your Web App

7. Go to the 'Application Settings' blade, scroll down until you see 'App settings' and add an entry (to use whichever port you setup), and click save:
```
Key:PORT
Value:5005
```

8. You should now be able to navigate to your Azure Web App address and see your project! If not - add '.scm' just before '.azurewebsites.net' in your URL e.g. http://ikdockergunicorn.scm.azurewebsites.net/ to access the Kudu console and go to 'Debug console' -> 'Bash', where you can access logfiles such as:
```
cd /home/LogFiles/docker
ls
```

9. The first load may take a while - this is because the docker image is downloaded to the WebApp. You can observe this by opening the first log-file in your directory:
```
cd /home/LogFiles/docker
cat docker_13_out.log
```
    The output should look like:
    ```
    7d27bd3d7fec: Verifying Checksum
    7d27bd3d7fec: Download complete
    7d27bd3d7fec: Pull complete
    44ae682c18a3: Pull complete
    824bd01a76a3: Pull complete
    68fe59875298: Pull complete
    9ca1d7ae0c4b: Pull complete
    46beba4b643f: Pull complete
    651cd581382c: Pull complete
    Digest: sha256:1efdaef9d8c208753fe36ccff197f28c719cc5f7d0bf5ff12f839f04e76c5f98
    Status: Downloaded newer image for ikmscontainer.azurecr.io/ilkarman/dockergunicorn:latest
    ```

In [163]:
!curl http://testdockik.azurewebsites.net/

healthy

In [164]:
!curl http://testdockik.azurewebsites.net/cntk

CNTK version: 2.0.beta15.0

In [168]:
jsonimg = url_img_to_json_img(IMAGEURL)
jsonimg[:100]  # Example of json strin

'{"input": "[\\"iVBORw0KGgoAAAANSUhEUgAAAOAAAADgCAIAAACVT/22AAEAAElEQVR4nMz9edRkyXUfBt4l4i2Z+a311b70vg'

In [169]:
headers = {'content-type': 'application/json'}
requests.post('http://testdockik.azurewebsites.net/api/uploader', 
              data=jsonimg,
              headers=headers).content

'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body bgcolor="white">\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.10.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'